In [12]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter # to print to tensorboard
from tqdm import tqdm

In [28]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1
        )
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1
        )
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

In [29]:
# Set Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
#Hyperparameters
learning_rate = 0.001
in_channels = 1
num_classes = 10
batch_size = 64
num_epochs = 3

In [31]:
#load data
train_dataset = datasets.MNIST(
    root='dataset/',
    train=True,
    transform=transforms.ToTensor(),
    download=False
)
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_dataset = datasets.MNIST(
    root='dataset/',
    train=False,
    transform=transforms.ToTensor(),
    download=False
)
test_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [32]:
model = CNN(in_channels=in_channels,num_classes=num_classes).to(device=device)

In [33]:
#Loss and Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate, weight_decay=0.0)

In [34]:
model.train()
writer = SummaryWriter(f"runs/MNIST/tryingout_tensorboard")

In [35]:
step = 0
for epoch in range(num_epochs):
    losses = []
    accuracies = []
    for batch_idx, (data, targets) in tqdm(enumerate(train_loader)):
        data = data.to(device)
        targets = targets.to(device)

        #forward
        score = model(data)
        loss = loss_fn(score,targets)
        losses.append(loss.item())

        #backward
        optimizer.zero_grad()
        loss.backward()

        #optimizer step
        optimizer.step()

        #Calculate 'running accuracy'
        _,predictions = score.max(1)
        num_correct = (predictions==targets).sum()
        running_train_acc = float(num_correct)/float(data.shape[0])
        
        # plotting loss & acc to tensorboard
        writer.add_scalar('Training loss',loss,global_step=step)
        writer.add_scalar('Training accuracy:',running_train_acc,global_step=step)
        step+=1

938it [00:17, 53.62it/s]
938it [00:15, 59.03it/s]
938it [00:16, 57.47it/s]


In [41]:
# Now lets check the tensorboard using command: 'python -m tensorboard.main --logdir runs' in terminal